# Parallel Quicksort Performance Analysis
## Science Methodology, Confidence Interval & Performance Evaluation (solo work)
**Author:** Andrei Bituleanu

This experiment aims to evaluate the performance of a parallel quicksort implementation in C using Pthreads, compared against its sequential variant.

The primary objective is to understand:
- How the number of threads or array size impacts sorting performance.
- At what point parallelism provides a measurable speed-up.
- How thread management overhead affects runtime efficiency.

All tests were executed on a machine with an 8-core processor (2.90 GHz) using GCC (pthread) and analyzed in a Jupyter Notebook environment for visualization and computation reproducibility.

All results and visualizations in this report are generated from code cells executed below.

### All source code below comes from the parallelQuicksort.c file by Joshua Stough and Arnaud Legrand:

**https://gricad-jupyter.univ-grenoble-alpes.fr/hub/user-redirect/lab/tree/Science-Methodology/exercice1/quicksort.ipynb**

Below is a sanity test to make sure everything works as intended. As you can see, by default, the sequential is significatively more efficient than its parallel counterpart.

%cd /home/bituleaa/notebooks/Science-Methodology/exercice1
!./M2R-ParallelQuicksort/src/parallelQuicksort 1000000

After verifying the program’s correctness with a baseline test (`N = 1,000,000`), I extended the analysis to study how parallel quicksort performance scales when varying the array size with larger numbers.

Initially, I experimented with increasing array sizes defined by `N = 10,000,000 × i`, where `i` is the iteration number.  
At this stage, the number of threads was constant (THREAD_LEVEL = 10).  
The results revealed that the sequential quicksort consistently and often narrowly outperformed the parallel version at 10 threads.

This indicates that the limitation does not lie in the idea of parallelism itself, but rather in its implementation strategy.

To document the first scaling experiment, where I varied the array size while keeping the thread count constant to 10,  
the following code snippet outlines the procedure used to collect runtimes and confidence intervals.

First start by running the next cell once, if you encounter an error trying to run any other code that will come after, 
that will compile the file. Then you may run the code snippet below.

In [ ]:
!gcc -O2 -pthread -DTHREAD_LEVEL=10 M2R-ParallelQuicksort/src/parallelQuicksort.c -o M2R-ParallelQuicksort/src/parallelQuicksort
!chmod +x M2R-ParallelQuicksort/src/parallelQuicksort

In [3]:
# Run this code snippet to reproduce my analysis.

import subprocess
import pandas as pd

exe_path = "./M2R-ParallelQuicksort/src/parallelQuicksort"

# 5 repetitions per test to ensure a confidence interval
repetitions = 5
results = []

for i in range(1, 3):  # N = 10M, 20M, 30M
    N = 10_000_000 * i
    print(f"\n=== Running test with N = {N:,} ===")
    for r in range(repetitions):
        result = subprocess.run([exe_path, str(N)], capture_output=True, text=True)
        output = result.stdout.strip()
        print(f"Run {r+1}:")
        print(output)
        results.append({"N": N, "output": output})

df_raw = pd.DataFrame(results)
df_raw.to_csv("parallel_quicksort_raw.csv", index=False)
print("Results saved to parallel_quicksort_raw.csv")


=== Running test with N = 10,000,000 ===
Run 1:
Thread value = 10 
Sequential quicksort took: 1.575684 sec.
Parallel quicksort took: 2.693161 sec.
Built-in quicksort took: 4.141856 sec.
Run 2:
Thread value = 10 
Sequential quicksort took: 1.443216 sec.
Parallel quicksort took: 2.951239 sec.
Built-in quicksort took: 4.160295 sec.
Run 3:
Thread value = 10 
Sequential quicksort took: 1.495248 sec.
Parallel quicksort took: 2.898793 sec.
Built-in quicksort took: 4.175829 sec.
Run 4:
Thread value = 10 
Sequential quicksort took: 1.482642 sec.
Parallel quicksort took: 2.941967 sec.
Built-in quicksort took: 4.178306 sec.
Run 5:
Thread value = 10 
Sequential quicksort took: 1.464856 sec.
Parallel quicksort took: 3.109609 sec.
Built-in quicksort took: 4.169508 sec.

=== Running test with N = 20,000,000 ===
Run 1:
Thread value = 10 
Sequential quicksort took: 3.072275 sec.
Parallel quicksort took: 5.216670 sec.
Built-in quicksort took: 8.679066 sec.
Run 2:
Thread value = 10 
Sequential quicksort

From the results, it becomes clear that regardless of the array size, the sequential version consistently achieves better runtimes than the parallel one.

**You may want to view the CSV file that it generated.**
    
Since the increasing the array size doesn't make any notable change to the efficiency of both algorithm's ratio, we can exclude the risk of
a Simpson paradox and move onto the next experiment where the thread count will change and the array size will remain constant, at an arbitrarily high array size.

Experimental Direction 2 : Changing Thread levels at a constant N array size

After confirming that increasing array size does not make the parallel quicksort outperfmorm the sequential one,
the next step is to investigate how the number of threads (THREAD_LEVEL) influences performance for a fixed, large array size.

In this experiment:

The array size N is kept constant at 10 000 000 elements.

The number of threads is varied across the values: 1, 2, 4, 6, 8, 10.

All results are saved automatically to a CSV for reproducibility and reused as is in my D3.js visualization.

The goal is to determine whether increasing thread depth ever leads to performance gains,
or whether synchronization and thread management overhead remain dominant even at large workloads.

Each configuration is executed 5 times to estimate a variability that computes a 95% confidence interval on the mean runtime.

For each run we store sequential, parallel and built-in quicksort. For each group we compute the mean execution time, and a 95% confidence interval around the mean.

You may now run the code snippet below to reproduce the results for yourself:

In [6]:
# Run this code if needed to test again the results of 5 repetitions alongside the calculation of the CI computation on your machine.
# If desired, use your new CSV file generated and replace it in repo you will have cloned to your machine to run its D3.js visualization.

import pandas as pd
import re
import numpy as np
from scipy.stats import t

raw = pd.read_csv("parallel_quicksort_threads_raw.csv")

def extract_times(out: str):
    seq = re.search(r"Sequential quicksort took:\s*([0-9.eE+-]+)\s*sec\.", out)
    par = re.search(r"Parallel quicksort took:\s*([0-9.eE+-]+)\s*sec\.", out)
    bi  = re.search(r"Built-in quicksort took:\s*([0-9.eE+-]+)\s*sec\.", out)
    return pd.Series({
        "time_seq": float(seq.group(1)),
        "time_par": float(par.group(1)),
        "time_builtin": float(bi.group(1)),
    })

df = pd.concat(
    [raw.drop(columns=["Output"]), raw["Output"].astype(str).apply(extract_times)],
    axis=1
)

def mean_ci95(x: pd.Series):
    x = x.to_numpy()
    n = len(x)
    mean = x.mean()
    std = x.std(ddof=1)
    half = t.ppf(0.975, n - 1) * std / np.sqrt(n)
    return pd.Series({"mean": mean, "ci_low": mean - half, "ci_high": mean + half})

seq = df.groupby(["Thread_Level", "N"])["time_seq"].apply(mean_ci95).unstack().add_prefix("seq_")
par = df.groupby(["Thread_Level", "N"])["time_par"].apply(mean_ci95).unstack().add_prefix("par_")
bi  = df.groupby(["Thread_Level", "N"])["time_builtin"].apply(mean_ci95).unstack().add_prefix("builtin_")

summary_min = seq.join(par).join(bi).reset_index()

summary_min.to_csv("parallel_quicksort_threads_ci_min.csv", index=False)

summary_min


,Thread_Level,N,seq_mean,seq_ci_low,seq_ci_high,par_mean,par_ci_low,par_ci_high,builtin_mean,builtin_ci_low,builtin_ci_high
0,1,10000000,1.493698,1.434769,1.552627,1.629095,1.494983,1.763207,4.165114,4.055896,4.274332
1,2,10000000,1.508157,1.419384,1.596931,1.824345,1.660412,1.988278,4.189956,4.083794,4.296118
2,4,10000000,1.500801,1.459204,1.542398,2.333883,1.686856,2.980910,4.179431,4.116147,4.242715
3,6,10000000,1.503671,1.470038,1.537304,2.760795,2.380700,3.140890,4.172008,4.117593,4.226423
4,8,10000000,1.480241,1.444348,1.516133,2.663174,2.360561,2.965788,4.170254,4.050492,4.290015
5,10,10000000,1.463958,1.428650,1.499267,2.714815,2.433962,2.995668,4.177069,4.114171,4.239967


I brought a small adjustment to the code by removing the #define THREAD_LEVEL 1 for the lines below so I wouldn't get an error when changing the Thread level for every new test run:

`ifndef THREAD_LEVEL`
`define THREAD_LEVEL 1`
`endif`

I invite you, if needed, to admire the table in the file generated from the code snippet, but for a more complete
visualisation of our results, below is the d3.js graph of the code we just ran, reproduction of the test should globally
be very similar as only the machine could make a slight influence in absolute results, but the graph should be virtually
the same. The code is always accessible if need on this Github repo at :
**Science-Methodology/exercice1/M2R-ParallelQuicksort/src/parallelQuicksort.c**

Anyway, let's observe our results.

The link to the html page showing off the graph is the following since there's boxes to read when hovering over a point:

https://starplatinumsanfr.github.io/Quicksort-Analysis/

![threadlevel](threadlevel.png)




First of all, each point in the mean runtime has 5 repetitions. The cloudy area with the whiskers serve as the 95% confidence interval. The CI was computed offline, on a GA502I Asus
laptop model without using the charging port. We used a discrete parameter for the Thread levels so any cloud or potential value observed outside of a specific x value is out of scope and irrelevant
except if we wanna look at the variance visually.

**Built-in algorithm** : We are gonna pass by this one fast, but it seems like the built-in algorithm remains pretty stable and consistent regardless of the thread count, standing as
the slowest runtime by far.

**Sequential algorithm** : This one seems to be clearly the best and most efficient algorithm regardless of the thread level for N = 10 000 000, the only time it COULD be advantageous
to use parallelism is when we only use 1 thread, which could imply that a single thread level is easier to manage for paralellism when the count is only in the tens of millions. However,
we can notice that the sequential algorithm seems to be the most unstable when the thread level = 2 as the variability of runtimes over 5 tests varies the most between the min and max,
but no where near as much as the parallel algorithm.

**Parallel algorithm** : This quicksort algorithm has a much wider CI, especially starting at 4 threads, but the variability is the most extreme at 4 threads too. This could be due 
to the way parallelism uses non-determinism, by using thread scheduling, synchronization and memory contention. The highest variability suggests a less predictable performance
for N = 10 000 000. More threads would be required to draw such a conclusion, but the runtime seems to stabilise for the mean starting at 6 threads for parallelism.

For all thread levels except for THREAD_LEVEL = 1, the parallel and sequential CI do not overlap, which suggests that indeed the performance gap is statistically significant and it's
not due to noise. For such a count of items in an array, the thread management and synchronization costs are not worth it to replace sequentialism with paralellism. However, if 
stability is what we desire, the built-in algorithm is better suited, yet much slower than the other two up to 10 threads.

We are now gonna proceed to do a linear regression to quantify the relationship we just stated. We are gonna perform a linear regression on speedup as we are here only working with and analysing variance in thread levels.

We are still talking about N = 10 000 000 and below as seen in the first code snippet.

**The formula for speedup is pretty simple, it's basically the Speedup(p) = the mean runtime of the sequential algorithm over the mean runtime of the parallel algorithm for a given p thread level.**
We know that if the Speedup(p) is greater than 1, then parallelism is more efficient, if it's = 1 then we get no gain and finally lesser than 1 means parallelism is slower than the sequential algorithm.

With that formula, we will get a linear regression of Speedup(p) = α⋅p + β , where p is the thread level, α the speedup gained for each extra thread and β the intercept.
α > 0 would mean the slope being positive and adding threads will increase speedup.
a = 0 would mean we gain no speedup from an increased thread count.
α < 0 would mean we actually lose speed by increasing thread count.

In addition to the slope α and intercept β, we also report the coefficient of determination, R2 which measures how well the linear regression model explains the observed data.
The closer R2 is to 1, the more the data follows a straight line, the closer it is to 0, the more the model shows no linear relationship.

Here is what we got from applying those formulas with the data from the CSV table:

![linear-regression](regression.png)

The speedup linear regression confirms our earlier observations made with the confidence intervals. First all our observed values remain strictly below 1 for all tested thread levels which suggests that in fact an array size of N = 10 000 000 is too small for the parallelism algorithm implementation to be more worth it than the sequential one, in fact, it is consistently slower and only gets slower with a higher thread count. Therefore, adding more threads will not make the algorithm any better, but rather much worse. Finally, the R2 coefficient of determination shows that thread levels hold a pretty significant impact on the Quicksort algorithm even if it's not perfect, it shows that the impact is more of a loss in speedup and runtime rather than the opposite when adding more threads.

**In conclusion, despite using parallelism, the parallel quicksort implementation does not achieve a scalable performance for our workload. Confidence intervals computation reveal the high variability in execution times, while the linear regression on speedup confirms that increasing the number of threads does not improve performance in any way. This highlights that parallelism does not guarantee performance gains in our context.**